In [0]:
import os 
os.chdir("drive/ccks/")

In [1]:
! ls

adc.json  drive  sample_data


In [0]:
# ! wget https://github.com/KonroyZhu/utils_model/archive/master.zip
# ! mv master.zip entity_classifier
# ! cd entity_classifier && unzip master.zip
# ! cd entity_classifier && mv utils_model-master utils_model

In [0]:
! pip install pkuseg

     |████████████████████████████████| 50.2MB 1.1MB/s 


### RUN

In [0]:
import re

import gensim
import jieba.posseg as psg
import lightgbm as lgb
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold

from entity_classifier.utils import single_letter_process, get_loc, get_chunks, get_bi_chunks_loc, clean_ent, \
    get_bi_chunk_topic_sim
from entity_classifier.word2vec_doc2vec import get_sim

data = pd.read_csv("train_classifier.csv")
texts = data["text"]
topics = data["topic"]
entities = data["entity"]
labels = data["label"]






#### 特征工程

In [4]:
import json
with open("entity_classifier/freq/ent_top.json","r",encoding="utf-8") as f:
  ent_top= json.load(f)
with open("entity_classifier/freq/top_ent.json","r",encoding="utf-8") as f:
  top_ent= json.load(f)
print("done!")

done!


In [5]:
# 特征工程
w2v = gensim.models.KeyedVectors.load_word2vec_format('entity_classifier/utils_model/word2vecModel.bin', binary=True)
print("w2v loaded")


def get_features(texts, topics, entities, labels=None):
    # 从原始数据中获取特征
    X, y = [], []
    for text, topic, ent, label in zip(texts, topics, entities, labels):
        ent, text = clean_ent(ent, text)
        loc, text_len = get_loc(ent, text)  # 实体位置与句子长度
        chunks = get_chunks(text)  # 采用符号分句
        bi_chunks_loc, bi_num_chunk = get_bi_chunks_loc(ent, chunks)  # 提取分句数与包含实体的句子
        ent_len = len(ent)  # 实体长度
        # bi_chunk_topic_sim_w2v = get_bi_chunk_topic_sim(chunks=chunks,topic=topic,model=w2v)
        ent_topic_sim = get_sim(sentence1=ent, sentence2=topic, model=w2v)
        ent_text_sim = get_sim(sentence1=ent, sentence2=text, model=w2v)
        
        try:
          ent_top_rate = ent_top[ent][topic]
        except:
          ent_top_rate = 0.
        try:
          top_ent_rate = top_ent[topic][ent]
        except:
          top_ent_rate = 0.
        
        # topic_text_sim = get_sim(sentence1=topic, sentence2=text, model=w2v)

        # feat = [loc, text_len, bi_num_chunk, bi_chunks_loc]  # acc 0.5513821138211382
        # feat = [loc, text_len, bi_num_chunk, bi_chunks_loc, ent_len]  # acc 0.6222764227642277
        # feat = [loc, text_len, bi_num_chunk, bi_chunks_loc, ent_len, text_len / bi_num_chunk]  # 0.6260162601626016

        # feat = [loc, text_len, bi_num_chunk, bi_chunks_loc, ent_len,
        #         bi_num_chunk / text_len,bi_chunk_topic_sim_w2v] # 0.6248780487804878

        # feat = [loc, text_len, bi_num_chunk, bi_chunks_loc, ent_len,
        #        bi_num_chunk / text_len,ent_topic_sim] # 0.66130081300813

        # feat = [loc, text_len, bi_num_chunk, bi_chunks_loc, ent_len,
        #        bi_num_chunk / text_len, ent_topic_sim, ent_text_sim]  # 0.6635772357723577
        
#         feat = [loc, text_len, bi_num_chunk, bi_chunks_loc, ent_len,
#                 bi_num_chunk / text_len, ent_topic_sim, ent_text_sim,top_ent_rate]  # acc: 0.8739837398373984
        
        feat = [loc, text_len, bi_num_chunk, bi_chunks_loc, ent_len,
                bi_num_chunk / text_len, ent_topic_sim, ent_text_sim,top_ent_rate,]  
        


        print(feat)
        X.append(feat)  # 特征，标签
        y.append(label)
    return X, y
  
X, y = get_features(texts, topics, entities, labels)

w2v loaded
[34, 66, 3, 1, 2, 0.045454545454545456, 0.9470003, 0.94586813, 0.0005773672055427252]
[29, 66, 3, 1, 4, 0.045454545454545456, 0.98861825, 0.98798776, 0.0]
[6, 112, 31, 6, 2, 0.2767857142857143, 0.95367473, 0.99231094, 0.0]
[21, 112, 31, 2, 3, 0.2767857142857143, 0.9889601, 0.9597759, 0.003246753246753247]
[27, 46, 3, 1, 5, 0.06521739130434782, 0.926585, 0.9294637, 0.0003979307600477517]
[27, 46, 3, 1, 4, 0.06521739130434782, 0.9663108, 0.96701235, 0.0]
[63, 105, 31, 16, 12, 0.29523809523809524, 0.9982935, 0.99509263, 0.0]
[14, 105, 31, 20, 4, 0.29523809523809524, 0.9929675, 0.9994426, 0.008620689655172414]
[1, 62, 3, 1, 1, 0.04838709677419355, 0.9983923, 0.99649674, 0.0]
[5, 62, 3, 1, 4, 0.04838709677419355, 0.99956524, 0.9985203, 0.0011547344110854503]
[18, 62, 3, 1, 4, 0.04838709677419355, 0.9996186, 0.9986096, 0.0005773672055427252]
[58, 189, 255, 4, 6, 1.3492063492063493, 0.9698974, 0.9980363, 0.002]
[46, 189, 255, 4, 6, 1.3492063492063493, 0.9647587, 0.99667597, 0.0]
[1

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[0, 53, 3, 1, 4, 0.05660377358490566, 0.9988541, 0.998649, 0.0007541478129713424]
[34, 59, 3, 2, 4, 0.05084745762711865, 0.99669, 0.99827045, 0.0005473453749315818]
[29, 59, 3, 2, 1, 0.05084745762711865, 0, 0, 0.0]
[29, 59, 3, 2, 4, 0.05084745762711865, 0.9950331, 0.9967789, 0.0010946907498631637]
[0, 49, 1, 1, 1, 0.02040816326530612, 0, 0, 0.0]
[23, 49, 1, 1, 4, 0.02040816326530612, 0.99970007, 0.99927163, 0.00904977375565611]
[3, 52, 7, 1, 3, 0.1346153846153846, 0.99494475, 0.9961894, 0.0005473453749315818]
[0, 52, 7, 1, 2, 0.1346153846153846, 0.9852163, 0.9866793, 0.0]
[30, 57, 15, 12, 4, 0.2631578947368421, 0.9390557, 0.99910533, 0.009124087591240875]
[5, 57, 15, 5, 2, 0.2631578947368421, 0.9117474, 0.9987739, 0.0030413625304136255]
[17, 57, 15, 13, 2, 0.2631578947368421, 0.9571546, 0.9948302, 0.0006082725060827251]
[45, 63, 15, 4, 4, 0.23809523809523808, 0.9976044, 0.9967281, 0.0]
[45, 63, 15, 0, 11, 0.23809523809523808, 0.99754953, 0.9966847, 0.0]
[50, 63, 15, 8, 6, 0.23809523809

#### 模型训练

In [6]:
# 训练模型
# trianing
models = []
predictions = []
lgb_params = {'num_leaves': 31,
              'min_data_in_leaf': 32,
              'max_depth': -1,
              'learning_rate': 0.005,
              "min_child_samples": 20,
              "boosting": "gbdt",
              "feature_fraction": 0.9,
              "bagging_freq": 1,
              "bagging_fraction": 0.9,
              'n_estimators': 10000,
              "bagging_seed": 11,
              "metric": 'rmse',
              "lambda_l1": 0.1,
              "nthread": 50,
              "verbosity": 0}

X, y = np.array(X), np.array(y)
kf = KFold(n_splits=20, shuffle=False)
for fold_, (trn_, val_) in enumerate(kf.split(X)):
    trn_x, trn_y = X[trn_, :], y[trn_]  # , df_label1[trn_]
    val_x, val_y = X[val_, :], y[val_]  # , df_label1[val_]
    model = lgb.LGBMRegressor(**lgb_params)
    model.fit(trn_x, trn_y, eval_set=[(trn_x, trn_y), (val_x, val_y)], eval_metric='mae', verbose=50,
              early_stopping_rounds=250)
    p = model.predict(val_x)
    predictions.extend(p > 0.5)
    models.append(model)

Training until validation scores don't improve for 250 rounds.
[50]	training's l1: 0.438724	training's rmse: 0.441285	valid_1's l1: 0.445078	valid_1's rmse: 0.448211
[100]	training's l1: 0.39062	training's rmse: 0.399543	valid_1's l1: 0.402569	valid_1's rmse: 0.413461
[150]	training's l1: 0.351252	training's rmse: 0.368941	valid_1's l1: 0.3686	valid_1's rmse: 0.390139
[200]	training's l1: 0.320305	training's rmse: 0.347604	valid_1's l1: 0.342151	valid_1's rmse: 0.375307
[250]	training's l1: 0.293696	training's rmse: 0.331295	valid_1's l1: 0.319696	valid_1's rmse: 0.365279
[300]	training's l1: 0.27176	training's rmse: 0.319304	valid_1's l1: 0.301808	valid_1's rmse: 0.359478
[350]	training's l1: 0.254556	training's rmse: 0.310733	valid_1's l1: 0.287878	valid_1's rmse: 0.355873
[400]	training's l1: 0.240649	training's rmse: 0.304216	valid_1's l1: 0.277119	valid_1's rmse: 0.354062
[450]	training's l1: 0.228895	training's rmse: 0.298925	valid_1's l1: 0.268379	valid_1's rmse: 0.353034
[500]	

In [7]:
# eval 验证
from sklearn.metrics import accuracy_score

print(len(predictions), len(y))
print("acc:", accuracy_score(y_true=y, y_pred=predictions))

6150 6150
acc: 0.8739837398373984
